In [ ]:
import pandas as pd
from faker import Faker
import random
from datetime import timedelta
import pyodbc

# ----------------------------
# Data Generation Section
# ----------------------------

# Initialize Faker
fake = Faker()

# Define number of records (adjust as needed)
num_publishers = 100
num_branches = 10
num_books = 1000
num_students = 500
num_loans = 1500

# Generate publisher_details data
publishers = {}
while len(publishers) < num_publishers:
    pub_name = fake.company()
    if pub_name not in publishers:
        publishers[pub_name] = {
            'publisher_name': pub_name,
            'publisher_address': fake.address(),
            'contact_number': fake.numerify("###-###-####")  # Fixed phone format
        }
publisher_df = pd.DataFrame(list(publishers.values()))

# Generate library_branch_details data (simulate identity column)
branches = []
for i in range(num_branches):
    branches.append({
        'branch_id': i + 1,
        'branch_name': f'Branch {i+1}',
        'branch_address': fake.address()
    })
branches_df = pd.DataFrame(branches)

# Generate student_details data (simulate identity column)
students = []
for i in range(num_students):
    students.append({
        'student_id': f'ST{str(i+1).zfill(4)}',
        'first_name': fake.first_name(),
        'last_name': fake.last_name(),
        'email': fake.email()[:20],
        'phone_number': fake.numerify("###-###-####"),  # Fixed phone format
        'dept_name': fake.word().capitalize()
    })
students_df = pd.DataFrame(students)

# Generate book_details data (simulate identity column)
books = []
for i in range(num_books):
    pub = publisher_df.sample(1).iloc[0]
    branch = branches_df.sample(1).iloc[0]
    books.append({
        'book_id': i + 1,
        'title': fake.sentence(nb_words=4).rstrip('.'),
        'publisher_name': pub['publisher_name'],
        'branch_id': branch['branch_id'],
        'is_loaned': 0  # default: not loaned
    })
books_df = pd.DataFrame(books)

# Generate book_loaned_details data (simulate identity column)
loans = []
for i in range(num_loans):
    book = books_df.sample(1).iloc[0]
    student = students_df.sample(1).iloc[0]
    
    issue_date = fake.date_between(start_date='-1y', end_date='today')
    due_date = issue_date + timedelta(days=14)
    # 70% chance the book has been returned; otherwise, returned_date remains None
    if random.random() < 0.7:
        returned_date = issue_date + timedelta(days=random.randint(1, 20))
    else:
        returned_date = None

    loans.append({
        'loan_id': i + 1,
        'book_id': book['book_id'],
        'branch_id': book['branch_id'],
        'student_id': student['student_id'],
        'issue_date': issue_date,
        'due_date': due_date,
        'returned_date': returned_date
    })
loans_df = pd.DataFrame(loans)

# Generate borrower_details data based on loan records
borrower_groups = loans_df.groupby('student_id').size().reset_index(name='books_loaned')
borrowers = []
for idx, row in borrower_groups.iterrows():
    borrowers.append({
        'id': idx + 1,
        'student_id': row['student_id'],
        'due_amount': 0,  # Set to 0 for simplicity; modify if needed
        'books_loaned': row['books_loaned']
    })
borrowers_df = pd.DataFrame(borrowers)

# Optional: Save data to CSV files for inspection
publisher_df.to_csv("publisher_details.csv", index=False)
branches_df.to_csv("library_branch_details.csv", index=False)
books_df.to_csv("book_details.csv", index=False)
students_df.to_csv("student_details.csv", index=False)
loans_df.to_csv("book_loaned_details.csv", index=False)
borrowers_df.to_csv("borrower_details.csv", index=False)

print("Data generated and saved to CSV files.")

# ----------------------------
# Data Insertion Section (SQL Server)
# ----------------------------

# Update connection details with your own SQL Server configuration
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Myserver name;"        
    "DATABASE=YourDatabaseName;"
    "Trusted_Connection=yes;"     
)

# Establish connection
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Insert publisher_details
publisher_values = publisher_df[['publisher_name', 'publisher_address', 'contact_number']].values.tolist()
cursor.executemany(
    "INSERT INTO publisher_details (publisher_name, publisher_address, contact_number) VALUES (?, ?, ?)",
    publisher_values
)

# Insert library_branch_details (enable identity insert)
cursor.execute("SET IDENTITY_INSERT library_branch_details ON")
branch_values = branches_df[['branch_id', 'branch_name', 'branch_address']].values.tolist()
cursor.executemany(
    "INSERT INTO library_branch_details (branch_id, branch_name, branch_address) VALUES (?, ?, ?)",
    branch_values
)
cursor.execute("SET IDENTITY_INSERT library_branch_details OFF")

# Insert student_details
student_values = students_df[['student_id', 'first_name', 'last_name', 'email', 'phone_number', 'dept_name']].values.tolist()
cursor.executemany(
    "INSERT INTO student_details (student_id, first_name, last_name, email, phone_number, dept_name) VALUES (?, ?, ?, ?, ?, ?)",
    student_values
)

# Insert book_details (enable identity insert)
cursor.execute("SET IDENTITY_INSERT book_details ON")
book_values = books_df[['book_id', 'title', 'publisher_name', 'branch_id', 'is_loaned']].values.tolist()
cursor.executemany(
    "INSERT INTO book_details (book_id, title, publisher_name, branch_id, is_loaned) VALUES (?, ?, ?, ?, ?)",
    book_values
)
cursor.execute("SET IDENTITY_INSERT book_details OFF")

# Insert book_loaned_details (enable identity insert)
cursor.execute("SET IDENTITY_INSERT book_loaned_details ON")
loan_values = loans_df[['loan_id', 'book_id', 'branch_id', 'student_id', 'issue_date', 'due_date', 'returned_date']].values.tolist()
cursor.executemany(
    "INSERT INTO book_loaned_details (loan_id, book_id, branch_id, student_id, issue_date, due_date, returned_date) VALUES (?, ?, ?, ?, ?, ?, ?)",
    loan_values
)
cursor.execute("SET IDENTITY_INSERT book_loaned_details OFF")

# Insert borrower_details (enable identity insert)
cursor.execute("SET IDENTITY_INSERT borrower_details ON")
borrower_values = borrowers_df[['id', 'student_id', 'due_amount', 'books_loaned']].values.tolist()
cursor.executemany(
    "INSERT INTO borrower_details (id, student_id, due_amount, books_loaned) VALUES (?, ?, ?, ?)",
    borrower_values
)
cursor.execute("SET IDENTITY_INSERT borrower_details OFF")

# Commit transactions and close connection
conn.commit()
cursor.close()
conn.close()

print("Data successfully inserted into SQL Server.")


Data generated and saved to CSV files.


IntegrityError: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__publishe__8DBCD413E5A3DB42'. Cannot insert duplicate key in object 'dbo.publisher_details'. The duplicate key value is (Johnson PLC). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")